<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!mkdir ~/.kaggle
from google.colab import files
files.upload()
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d shusrith/betterer-preproc

Saving kaggle.json to kaggle.json
 98% 201M/206M [00:01<00:00, 132MB/s]
100% 206M/206M [00:01<00:00, 140MB/s]


In [ ]:
!unzip /content/betterer-preproc.zip

In [2]:
!kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

 94% 32.0M/34.1M [00:00<00:00, 57.8MB/s]
100% 34.1M/34.1M [00:00<00:00, 57.3MB/s]


In [ ]:
!unzip /content/alzheimers-dataset-4-class-of-images.zip

# PRE PROCESS
.JPG TO .NPY

In [4]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
            np.save(output_file_path, img_array)

input_folder_path = "/content/Alzheimer_s Dataset/train"
output_folder_path = "/kaggle/working/train"

jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [5]:
import os
import cv2
import numpy as np

def jpg_to_npy_in_folders(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, item in enumerate(os.listdir(input_folder)):
        item_path = os.path.join(input_folder, item)
        if os.path.isdir(item_path):
            jpg_to_npy_in_folders(item_path, output_folder)
        elif os.path.isfile(item_path) and item.endswith(".jpg"):
            img = cv2.imread(item_path)
            img_array = np.array(img)
            output_file_path = os.path.join(output_folder, f"{input_folder}{i}.npy")
            np.save(output_file_path, img_array)

# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)
jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [6]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)


5121

In [7]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 52 2560 1792


In [8]:
import imgaug.augmenters as iaa

# Define the augmenter
augmenter = iaa.Sequential([
    iaa.Multiply((0.8, 1.2)),
    iaa.LinearContrast((0.8, 1.2)),
])

augmented_images = []

for path in moderateDem:
    path = os.path.join("/kaggle/working/train", path)
    image = np.load(path)

    for _ in range(20):
        augmented_image = augmenter.augment_image(image)
        augmented_images.append(augmented_image)
augmented_images = np.array(augmented_images)

In [9]:
len(augmented_images)

1040

In [10]:
import cv2
import os


output_directory = "/kaggle/working/train"

os.makedirs(output_directory, exist_ok=True)

for i, image in enumerate(augmented_images):
    output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
    np.save(output_path, image)


print("Augmented images saved successfully.")


Augmented images saved successfully.


In [11]:
import os
import numpy as np
directory = '/kaggle/working/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

6161

In [12]:
mildDem = [i for i in files if i.startswith("mildDem")]
moderateDem = [i for i in files if "moderateDem" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "verymildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

717 1092 2560 1792


In [13]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/working/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
2    2560
3    1792
1    1092
0     717
Name: count, dtype: int64

In [14]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [15]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [16]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[1 3 0 ... 3 3 1]
[1 0 2 ... 3 3 2]


In [17]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(4928, 208, 176, 3) (1233, 208, 176, 3) (4928,) (1233,)


In [18]:
print(test_labels)
test_labels_encoded

[1 0 2 ... 3 3 2]


array([1, 0, 2, ..., 3, 3, 2])

In [19]:
print(x.index, x.values, x.array)

Index([2, 3, 1, 0], dtype='int64', name='Class') [2560 1792 1092  717] <PandasArray>
[2560, 1792, 1092, 717]
Length: 4, dtype: int64


In [20]:
s = x / len(files)
class_weights = {i:j for i, j in zip(s.index, s.values)}

class_weights
# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


{2: 0.4155169615322188,
 3: 0.2908618730725532,
 1: 0.17724395390358708,
 0: 0.11637721149164097}

In [28]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming you have loaded your MRI images into numpy arrays
# X is your image data, y are your labels

# Convert labels to one-hot encoding with 4 categories
train_labels_encoded = to_categorical(train_labels, num_classes=4)
test_labels_encoded = to_categorical(test_labels, num_classes=4)

# Calculate sample weights to handle imbalance
sample_weights = compute_sample_weight(class_weight='balanced', y=np.argmax(train_labels_encoded, axis=1))

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(208, 176, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 4 classes for Alzheimer's detection

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(train_images, train_labels_encoded,
                    steps_per_epoch=len(train_images) / 32,
                    epochs=50,
                    validation_data=(test_images, test_labels_encoded),
                    sample_weight=sample_weights)

Epoch 1/50
154/154 [==============================] - 11s 44ms/step - loss: 15.3462 - accuracy: 0.5310 - val_loss: 0.7022 - val_accuracy: 0.6707
Epoch 2/50
154/154 [==============================] - 7s 43ms/step - loss: 0.6654 - accuracy: 0.6660 - val_loss: 0.6738 - val_accuracy: 0.6796
Epoch 3/50
154/154 [==============================] - 6s 40ms/step - loss: 0.5353 - accuracy: 0.7350 - val_loss: 0.6429 - val_accuracy: 0.7161
Epoch 4/50
154/154 [==============================] - 7s 42ms/step - loss: 0.4433 - accuracy: 0.7764 - val_loss: 0.4965 - val_accuracy: 0.7875
Epoch 5/50
154/154 [==============================] - 7s 44ms/step - loss: 0.3419 - accuracy: 0.8330 - val_loss: 0.4304 - val_accuracy: 0.8159
Epoch 6/50
154/154 [==============================] - 6s 41ms/step - loss: 0.2633 - accuracy: 0.8748 - val_loss: 0.3002 - val_accuracy: 0.8848
Epoch 7/50
154/154 [==============================] - 6s 40ms/step - loss: 0.1941 - accuracy: 0.9083 - val_loss: 0.3152 - val_accuracy: 0.86

In [31]:
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)
y_prob = tf.nn.softmax(y_prob).numpy()

# Compute ROC AUC for each class
roc_auc = roc_auc_score(test_labels_encoded, y_prob, average='macro')

print("ROC AUC:", roc_auc)

39/39 [==============================] - 0s 10ms/step
ROC AUC: 0.9987596892400434


In [ ]:
# Save the model
model.save("my_cnn_model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
import os
import numpy as np
directory = '/content/Alzheimer_s Dataset/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1279

In [33]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [34]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/content/Alzheimer_s Dataset/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [35]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [37]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=4)
# Verify the encoded labels
print(labels.shape)

(1279, 4)


In [43]:
l = []
for i in labels:
  for j in range(4):
    if i[j] == 1:
      l.append(j)


In [ ]:
y_prob = model.predict(images)

# y_prob = tf.nn.softmax(y_prob).numpy()
# Get the class with the highest probability for each prediction
y_prob = np.argmax(y_prob, axis=1)  # Argmax gives index of max value
for i in range(len(labels)):
  print(l[i], y_prob[i])

In [45]:
from sklearn.metrics import classification_report

# Assuming you have predicted labels (predicted_classes) and true labels (test_labels_encoded)
report = classification_report(l, y_prob, output_dict=True)
for i in report:
  print(i, report[i])

0 {'precision': 0.8695652173913043, 'recall': 0.22346368715083798, 'f1-score': 0.3555555555555555, 'support': 179}
1 {'precision': 1.0, 'recall': 0.4166666666666667, 'f1-score': 0.5882352941176471, 'support': 12}
2 {'precision': 0.6036585365853658, 'recall': 0.928125, 'f1-score': 0.731527093596059, 'support': 640}
3 {'precision': 0.7090163934426229, 'recall': 0.3861607142857143, 'f1-score': 0.5, 'support': 448}
accuracy 0.6348709929632526
macro avg {'precision': 0.7955600368548232, 'recall': 0.4886040170258048, 'f1-score': 0.5438294858173154, 'support': 1279}
weighted avg {'precision': 0.6814956853713625, 'recall': 0.6348709929632526, 'f1-score': 0.5964664643278609, 'support': 1279}


In [ ]:
model.evaluate(images, labels)

40/40 [==============================] - 1s 22ms/step - loss: 2.8566 - sparse_categorical_accuracy: 0.4949 - f1_score: nan


[2.8566365242004395, 0.4949178993701935, nan]